In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [16]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns=['species'])
df = pd.concat([X, y], axis=1)

In [17]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
scaler = StandardScaler()

In [21]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

In [29]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [31]:
y_train_tensor.shape

torch.Size([120, 1])

In [40]:
class IrisNN(nn.Module):
  def __init__(self):
    super(IrisNN, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(4, 16),
        nn.ReLU(),
        nn.Linear(16, 3),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
    return self.model(x)

In [41]:
model = IrisNN()

In [42]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [43]:
y_train_labels = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long)
y_test_labels = torch.tensor(np.argmax(y_test, axis=1), dtype=torch.long)

In [48]:
epochs = 1000
for epoch in range(epochs):
  model.train()
  y_pred = model(X_train_tensor)
  loss = loss_fn(y_pred, y_train_labels)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch%100 == 0:
    print(f'Loss at epoch {epoch}: {loss.item():.4f}')

Loss at epoch 0: 0.5516
Loss at epoch 100: 0.5515
Loss at epoch 200: 0.5515
Loss at epoch 300: 0.5515
Loss at epoch 400: 0.5515
Loss at epoch 500: 0.5515
Loss at epoch 600: 0.5515
Loss at epoch 700: 0.5515
Loss at epoch 800: 0.5515
Loss at epoch 900: 0.5515


In [49]:
model.eval()

IrisNN(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=3, bias=True)
    (3): Softmax(dim=1)
  )
)

In [50]:
with torch.no_grad():
  y_test_pred = model(X_test_tensor)
  predictions = torch.argmax(y_test_pred, dim=1)
  accuracy = (predictions == y_test_labels).float().mean()
  print(f"\nTest Accuracy: {accuracy.item():.2f}")


Test Accuracy: 1.00
